In [28]:
import pandas as pd

In [3]:
ped = pd.read_csv("köln-schildergasse (west)-20180430-20200131-hour.csv", sep = ";")
holi = pd.read_csv("Feiertage_2018.01.01_2020.01.31.csv", sep = ";")
print(ped.head(3))
print(holi.head(3))



                     location        time of measurement  weekday  \
0  Schildergasse (West), Köln  2018-05-01 00:00:00 +0200  Tuesday   
1  Schildergasse (West), Köln  2018-05-01 01:00:00 +0200  Tuesday   
2  Schildergasse (West), Köln  2018-05-01 02:00:00 +0200  Tuesday   

   pedestrians count  temperature in ºc    weather condition  incidents  
0                  0                8.0  partly-cloudy-night        NaN  
1                  0                7.0  partly-cloudy-night        NaN  
2                146                7.0  partly-cloudy-night        NaN  
         date        name
0  2018-01-01     Neujahr
1  2018-03-30  Karfreitag
2  2018-04-01      Ostern


In [4]:
#Splitting up the Time of measurement into date and time
ped[["date", "time"]] = ped["time of measurement"].str.split(" ", 1, expand=True)
#Drpo Time of measuremnt, because it is split now and drop location
ped = ped.drop(["time of measurement", "location"], axis=1)
#Drop incidents, if only nan
ped.dropna(how='all', axis=1, inplace=True)

# restructure by date
ped = ped[ ["date", "time"] + [ col for col in ped.columns if col != "date" and col!= "time"] ]
#change name
holi = holi.rename(columns={"name": "holiday"})
#merge on date
ped = pd.merge(ped, holi, how = "left")

#put hollyday as 1 and no holyday as 0
ped["holiday"] = ped["holiday"].fillna(0)
ped["holiday"] = ped["holiday"].where(ped["holiday"] == 0, 1)

# clear time from the minute variation
ped["time"] = ped["time"].str.slice_replace(8, 16)

# Get a better insight into the weather data and missing data
print(ped["weather condition"].unique())
print(ped.isnull().sum() / ped.shape[0])

#See where the data are missing
null_data = ped[ped.isnull().any(axis=1)]
print(null_data)

['partly-cloudy-night' 'partly-cloudy-day' 'clear-night' 'clear-day'
 'rain' 'cloudy' 'fog' nan 'wind']
date                 0.000000
time                 0.000000
weekday              0.000000
pedestrians count    0.000000
temperature in ºc    0.001755
weather condition    0.001755
holiday              0.000000
dtype: float64
            date      time    weekday  pedestrians count  temperature in ºc  \
1086  2018-06-15  06:00:00     Friday                195                NaN   
1128  2018-06-17  00:00:00     Sunday                386                NaN   
1199  2018-06-19  23:00:00    Tuesday                233                NaN   
1236  2018-06-21  12:00:00   Thursday               5895                NaN   
1260  2018-06-22  12:00:00     Friday               5553                NaN   
1266  2018-06-22  18:00:00     Friday               7363                NaN   
1291  2018-06-23  19:00:00   Saturday               5852                NaN   
1319  2018-06-24  23:00:00     Sunday  

C:\Users\Biebert\AppData\Local\Temp\ipykernel_22444\1063994339.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  ped[["date", "time"]] = ped["time of measurement"].str.split(" ", 1, expand=True)


In [5]:
#because the data for temperature and weather condition are both missing, this can be dropped, due to the huge sample size
ped.dropna(inplace=True)


In [8]:
#Import datapackages
from datetime import date, timedelta
from datetime import datetime

#Import Dataframe from the holidays
school_holi = pd.read_csv("OpenData_Ferientermine.csv", sep = ";")
days_df_list = []

#Create dataframe with all dates singular listed 
for i in range(school_holi.shape[0]):
    #Get the start and enddate from the holidays
    start_date = datetime.strptime(school_holi["ErsterTagDate"][i], '%d.%m.%Y').date()
    end_date = datetime.strptime(school_holi["LetzterTagDate"][i], '%d.%m.%Y').date()

    #create dataframes out of the range and append to a list
    delta = end_date - start_date
    days = [start_date + timedelta(days=n) for n in range(delta.days + 1)]
    days_df = pd.DataFrame (days, columns = ['date'])
    days_df_list.append(days_df)

#create single dataframe out of the list and add value 1 for holiday
new_holi = pd.concat(days_df_list).reset_index(drop=True)
new_holi["school holiday"] = 1

#merg of the two frames
ped = pd.merge(ped, new_holi, how = "left")
ped["school holiday"] = ped["school holiday"].fillna(0)
ped






,date,time,weekday,pedestrians count,temperature in ºc,weather condition,holiday,school holiday
0,2018-05-01,00:00:00,Tuesday,0,8.0,partly-cloudy-night,1,0.0
1,2018-05-01,01:00:00,Tuesday,0,7.0,partly-cloudy-night,1,0.0
2,2018-05-01,02:00:00,Tuesday,146,7.0,partly-cloudy-night,1,0.0
3,2018-05-01,03:00:00,Tuesday,125,7.0,partly-cloudy-night,1,0.0
4,2018-05-01,04:00:00,Tuesday,84,6.0,partly-cloudy-night,1,0.0
...,...,...,...,...,...,...,...,...
15353,2020-01-31,19:00:00,Friday,5422,11.0,partly-cloudy-night,0,0.0
15354,2020-01-31,20:00:00,Friday,2691,11.0,partly-cloudy-night,0,0.0
15355,2020-01-31,21:00:00,Friday,1329,10.0,partly-cloudy-night,0,0.0
15356,2020-01-31,22:00:00,Friday,915,11.0,cloudy,0,0.0


# API

In [7]:
future_school_holi = pd.read_csv("future_school_holiday.csv", sep = ";")
future_school_holi


,Ferien,Erster Ferientag,Letzter Ferientag
0,Sommer,"Montag, 27. Juni 2022","Dienstag, 9. August 2022"
1,Herbst,"Dienstag, 4. Oktober 2022","Samstag, 15. Oktober 2022"
2,Weihnachten,"Freitag, 23. Dezember 2022","Freitag, 6. Januar 2023"
3,Ostern,"Montag, 3. April 2023","Samstag, 15. April 2023"
4,Pfingsten,"Dienstag, 30. Mai 2023","Dienstag, 30. Mai 2023"
5,Sommer,"Donnerstag, 22. Juni 2023","Freitag, 4. August 2023"
6,Herbst,"Montag, 2. Oktober 2023","Samstag, 14. Oktober 2023"
7,Weihnachten,"Donnerstag, 21. Dezember 2023","Freitag, 5. Januar 2024"
8,Ostern,"Montag, 25. Maerz 2024","Samstag, 6. April 2024"
9,Pfingsten,"Dienstag, 21. Mai 2024","Dienstag, 21. Mai 2024"


In [35]:
from datetime import date, timedelta

#Give input year (needs to be accesible form outside the code!!!!!!!!!!!!!!!!)
intput_year = 2022

#Formular for calculating easter sunday
k = intput_year // 100
m = 15 + (3 * k + 3) // 4 - (8 * k + 13) // 25
s = 2 - (3*k + 3) // 4
a = intput_year % 19

d = (19* a + m) % 30
r = (d + a // 11) // 29
og = 21 + d - r
sz = 7 - (intput_year + intput_year // 4 + s) % 7

oe =  7 - (og - sz) % 7

#days for easter sunday after teh first march (because daytime starts at first minus 1 again)
os = og + oe -1
first_march = date(intput_year, 3, 1)

# calculating the date for easter sunday
os_date = first_march + timedelta(days=os)


# The flexible holidays calculated from easter sunday
karfreitag = os_date - timedelta(days=2)
os_date = os_date
easter_monday = os_date + timedelta(days=1)
ascension = os_date + timedelta(days=39)
white_monday = os_date + timedelta(days=50)
corpus_christ = os_date + timedelta(days=60)

# The fixed holidays (for nrw)
new_year = date(intput_year, 1, 1)
day_of_work = date(intput_year, 5, 1)
germany_united = date(intput_year, 10, 3)
saints_day = date(intput_year, 11, 1)
first_christmas = date(intput_year, 12, 25)
second_christmas =date(intput_year, 11, 26)

# Creating dataframe with holidays
holiday_list_future = [new_year, karfreitag, os_date, easter_monday, day_of_work, ascension, white_monday, corpus_christ, germany_united, saints_day, first_christmas, second_christmas]
future_holidays = pd.DataFrame ({'date': holiday_list_future,
                                'holiday': [1 for i in range(len(holiday_list_future))]})
future_holidays

,date,holiday
0,2022-01-01,1
1,2022-04-15,1
2,2022-04-17,1
3,2022-04-18,1
4,2022-05-01,1
5,2022-05-26,1
6,2022-06-06,1
7,2022-06-16,1
8,2022-10-03,1
9,2022-11-01,1
